In [5]:
# utils
import pandas as pd
import os
import numpy as np
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import ast
import re
from scipy.stats import ttest_rel

# evaluation 
from evaluate import load

In [6]:
# open files in generations directory
def open_files(directory):
    files = os.listdir(directory)
    files.sort()
    return files

In [7]:
models_generations = {
    'en': {},
    'it': {},
    'ru': {},
    'ge': {}
}

files = open_files('generations')
for file in files:
    model = re.sub(r"^fine-tuned-|-exp2-gen\d+.csv", "", file)
    gen = re.search(r"gen\d+", file).group()

    model_generations = pd.read_csv('generations/' + file)
    # convert string to list
    model_generations['actual'] = model_generations['actual'].apply(ast.literal_eval)
    
    # if containt -en-
    if '-en-' in file:
        if model not in models_generations['en']:
            models_generations['en'][model] = {}
        models_generations['en'][model][gen] = model_generations
        print(f"model: {model} - gen: {gen}, lang: en")
    elif '-it-' in file:
        if model not in models_generations['it']:
            models_generations['it'][model] = {}
        models_generations['it'][model][gen] = model_generations
        print(f"model: {model} - gen: {gen}, lang: it")
    elif '-ge-' in file:
        if model not in models_generations['ge']:
            models_generations['ge'][model] = {}
        models_generations['ge'][model][gen] = model_generations
        print(f"model: {model} - ge: {gen}, lang: ge")
    elif '-ru-' in file:
        if model not in models_generations['ru']:
            models_generations['ru'][model] = {}
        models_generations['ru'][model][gen] = model_generations
        print(f"model: {model} - gen: {gen}, lang: ru")

model: Llama-3.1-8B-Instruct-en - gen: gen0, lang: en
model: Llama-3.1-8B-Instruct-en - gen: gen1, lang: en
model: Llama-3.1-8B-Instruct-en - gen: gen2, lang: en
model: Llama-3.1-8B-Instruct-ge - ge: gen0, lang: ge
model: Llama-3.1-8B-Instruct-ge - ge: gen1, lang: ge
model: Llama-3.1-8B-Instruct-ge - ge: gen2, lang: ge
model: Llama-3.1-8B-Instruct-it - gen: gen0, lang: it
model: Llama-3.1-8B-Instruct-it - gen: gen1, lang: it
model: Llama-3.1-8B-Instruct-it - gen: gen2, lang: it
model: Llama-3.1-8B-Instruct-ru - gen: gen0, lang: ru
model: Llama-3.1-8B-Instruct-ru - gen: gen1, lang: ru
model: Llama-3.1-8B-Instruct-ru - gen: gen2, lang: ru
model: Mistral-Nemo-Instruct-2407-en - gen: gen0, lang: en
model: Mistral-Nemo-Instruct-2407-en - gen: gen1, lang: en
model: Mistral-Nemo-Instruct-2407-en - gen: gen2, lang: en
model: Mistral-Nemo-Instruct-2407-ge - ge: gen0, lang: ge
model: Mistral-Nemo-Instruct-2407-ge - ge: gen1, lang: ge
model: Mistral-Nemo-Instruct-2407-ge - ge: gen2, lang: ge
mode

## Valutazione automatica

### <a href="https://huggingface.co/spaces/evaluate-metric/chrf">chrF++</a>

In [8]:
chrf = load("chrf")

Using the latest cached version of the module from C:\Users\OliverioM\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--chrf\d244bab9383988714085a8dacc4871986d9f025398581c33d6b2ee22836b4069 (last modified on Wed Sep  4 18:15:30 2024) since it couldn't be found locally at evaluate-metric--chrf, or remotely on the Hugging Face Hub.


In [9]:
chrf_scores = {'en': {}, 'it': {}, 'ge': {}, 'ru': {}}

for lang in models_generations:
    for model in models_generations[lang]:
        model_chrf_scores = []
        print(f'Language: {lang}, Model: {model}')
        
        for gen in sorted(models_generations[lang][model]):  # Ensure order (gen0, gen1, gen2)
            references = models_generations[lang][model][gen]['actual']
            predictions = models_generations[lang][model][gen]['prediction']

            gen_chrf_scores = []
            for i in tqdm(range(len(references)), desc="Calcolo punteggi CHRF"):
                reference_scores = []
                reference = references[i]
                for j in range(len(reference)):
                    actual = reference[j]
                    result = chrf.compute(predictions=[predictions[i]], references=[actual], word_order=2)
                    reference_scores.append(result['score'])
                
                gen_chrf_scores.append(np.mean(reference_scores))
            
            model_chrf_scores.append(gen_chrf_scores)

        # Store the three chrf scores instead of their mean
        chrf_scores[lang][model] = model_chrf_scores
        print(f'chrf scores for {model} ({lang}): {model_chrf_scores}')
        print(f'Average: {np.mean(model_chrf_scores)}')
        print()

Language: en, Model: Llama-3.1-8B-Instruct-en


Calcolo punteggi CHRF: 100%|██████████| 383/383 [00:07<00:00, 48.28it/s]


chrf scores for Llama-3.1-8B-Instruct-en (en): [[87.17181709025928, 78.93906724984905, 36.02861577538228, 60.08120395338609, 50.060800844227785, 69.47318157856837, 59.465087470690875, 54.66564061488424, 88.37126587816316, 76.89061603620279, 67.5773304806614, 62.89789277026777, 86.00472185077967, 77.12689871891617, 77.71215551645194, 66.12298560474608, 74.12903610779017, 64.60444886017625, 67.80152117726146, 84.79206641167995, 66.44696180942172, 74.25903036988984, 59.02034335031275, 86.10156813500487, 83.87423052011248, 70.953839607417, 83.27156861678628, 59.05698801844636, 71.86643013289985, 48.81388195570608, 76.79048235864246, 79.04694982064677, 80.19801630138143, 56.721757636055315, 69.87441324186756, 73.70975369339567, 66.98911873551886, 67.78768784068583, 95.37703512463582, 80.2281348281053, 65.9098715357725, 73.32762767074003, 67.52057170525717, 63.36980939735961, 77.20184524543548, 70.40289366581709, 80.61808151136682, 75.25040647803462, 74.95570456152299, 73.4908871202142, 86.7

Calcolo punteggi CHRF: 100%|██████████| 383/383 [00:09<00:00, 39.11it/s]


chrf scores for Mistral-Nemo-Instruct-2407-en (en): [[87.17181709025928, 78.93906724984905, 36.02861577538228, 61.41300393636724, 49.076174534382204, 72.42871118802584, 58.718437582861156, 57.82209080952136, 88.37126587816316, 76.89061603620279, 66.84439454162246, 67.77524843732421, 86.00472185077967, 56.60728390379876, 77.71215551645194, 66.12298560474608, 74.12903610779017, 64.60444886017625, 52.880232024903044, 83.39053033875025, 61.69563981022037, 75.41056430567656, 67.96588698801779, 87.21847037644379, 75.16186787094922, 70.953839607417, 83.27156861678628, 64.0766698228926, 68.06033885876751, 48.81388195570608, 60.04149705540827, 76.54154608185866, 80.13127735348887, 56.04278337987936, 69.87441324186756, 70.88450132283306, 67.09425722747162, 79.70431231526958, 91.78865289137383, 75.0578142341912, 66.1206458012178, 73.32762767074003, 63.682275969409545, 57.04653356722225, 73.29212504941073, 72.06503477243666, 80.61808151136682, 80.98627048608215, 74.95570456152299, 73.4908871202142

...

chrf scores for Llama-3.1-8B-Instruct-ge (ge): [[87.9246067447163, 88.6545944358612, 37.417414348695274, 54.051645520282854, 40.5155226303797, 64.15921147219392, 61.131469393048675, 52.5149725744244, 48.57190703677453, 82.54351295769524, 57.90245213761702, 67.0107531401945, 89.643746666902, 81.69056932382743, 64.77740910315778, 27.3200482620299, 36.63105352234357, 72.48654045790052, 65.74015902817095, 78.16863709193937, 66.34509985630432, 75.72838180692456, 48.07778029020009, 69.25285044561909, 71.33923319955638, 82.93251531012726, 76.73613181339508, 41.74090479446002, 66.4677224289715, 45.42898098990588, 57.18225084914874, 74.4952998491126, 77.74559075046191, 57.31135561444029, 55.73612655464569, 64.51512086190051, 52.92769045501708, 73.70236228847754, 100.0, 75.49324360127562, 51.919419716563475, 76.84462161171494, 49.44662800401769, 40.94491286238631, 73.4533078176732, 70.98907120857575, 71.45708491815289, 80.09178643791296, 76.32795848054126, 66.3890007956863, 85.4381187501099, 68.

Calcolo punteggi CHRF: 100%|██████████| 383/383 [00:09<00:00, 41.61it/s]


chrf scores for Mistral-Nemo-Instruct-2407-ge (ge): [[87.9246067447163, 88.6545944358612, 33.58604218006884, 54.051645520282854, 45.120904411422245, 61.67477022403838, 61.47433132199469, 42.0221549082136, 46.728046154468174, 84.42765026541316, 54.89750008781506, 64.87993817793071, 75.14119793797914, 49.86867522967188, 64.77740910315778, 27.3200482620299, 36.63105352234357, 59.27297313491564, 68.90224727084144, 82.45050600621092, 62.692001293405035, 79.63569695739726, 44.58436526378904, 77.09310912267154, 63.233332486138686, 65.10154873147896, 76.73613181339508, 45.807315398436764, 64.12887678913303, 49.98644468706207, 57.18225084914874, 72.43270311477627, 77.81312262925945, 54.62335891039544, 65.53725127276094, 57.52876174854072, 55.563564123519086, 67.07868503737127, 100.0, 75.50550692736144, 51.63930220379004, 71.44560483459848, 58.22871752560299, 50.17956013222417, 70.81014181486488, 57.53596463492189, 67.56094406166928, 48.64973190572536, 76.32795848054126, 75.74485632685195, 85.43

Calcolo punteggi CHRF: 100%|██████████| 383/383 [00:07<00:00, 50.19it/s]

chrf scores for Qwen2.5-7B-Instruct-ge (ge): [[87.9246067447163, 94.33553775579813, 37.03080000020097, 54.882463454143625, 48.11331795572613, 57.331692853829644, 53.42708505598988, 43.18805043521502, 48.57190703677453, 78.9900415137656, 55.93678537422488, 64.85233153901841, 92.71817503317129, 61.4072876275497, 45.52376799374142, 24.565232841986614, 37.32089125092825, 72.44105222606844, 54.00870026944364, 82.45050600621092, 58.79143881495069, 67.51633968666538, 47.056512942237084, 57.35941870560414, 51.44277911537788, 82.93251531012726, 76.73613181339508, 38.86803573996972, 61.29807212779094, 49.98644468706207, 46.67304753870799, 75.1722755768692, 77.642869576872, 53.29106503294245, 69.87074798231951, 49.0667518040181, 44.902213308611806, 62.623323341223845, 100.0, 80.25609523060079, 53.876887302848296, 76.38000352813087, 56.681003251641044, 41.467058161117556, 73.4533078176732, 57.973075868515416, 67.56094406166928, 49.38855690737707, 76.32795848054126, 64.50565239952492, 46.1897509014

In [11]:
import pickle

# Save chrF++ scores
with open('chrf_scores-exp2.pkl', 'wb') as f:
    pickle.dump(chrf_scores, f)